In [ ]:
import sys
sys.path.append('../')
import tensorflow as tf
import numpy as np
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from ipywidgets import interact

In [ ]:
from pydens import add_tokens, Solver, NumpySampler, cart_prod, ConstantSampler
from utils import plot_loss, plot_pair_1d, plot_2d, plot_sections_2d

## Introductory examples

To begin with, let us go through all of the steps of configuring a `PyDEns`-model using a simple example.

### The easiest example possible: first-order ordinary differential equation in $\mathcal{R}$
$$
\frac{d u}{d t}= 2\pi\cos[2 \pi t]; \quad t \in [0, 1],\ f(0)=1.
$$

The very first thing to do is to add a set of mathematical tokens to the current namespace:

In [ ]:
add_tokens()

Configuring `PyDENs`-model comes down to several step:

* setting up dimensionality of the problem:

In [ ]:
n_dims = 1

* describing differential form representing the equation using created tokens

In [ ]:
form = lambda u, t: D(u, t) - 2 * np.pi * cos(2 * np.pi * t)

* preparing initial/boundary conditions:

In [ ]:
initial_condition = 1 # alternatively, can be callable

* choosing point-sampling scheme

In [ ]:
s = NumpySampler('uniform')

in short, each `Sampler` is an entity that samples points: 

In [ ]:
s.sample(3)

we can now assemble a `configuration-dict` for the pde-problem

In [ ]:
pde = {'n_dims': n_dims,
       'form': form,
       'initial_condition': initial_condition}

* **[optional]** set up a neural network architecture using `layout`-string, that specifies the sequence of layers in a network using letters, like `f`(stands for fully connected) or `a` (stands for activation).

In [ ]:
body = {'layout': 'fa fa f',
        'units': [10, 25, 1],
        'activation': [tf.nn.tanh, tf.nn.tanh]}

we are all set to assemble the configuration-dict for the whole model

In [ ]:
config = {'body': body,
          'pde': pde}

..and initialize the model-instance

In [ ]:
model = Solver(config)

it is not ready yet: we still have to train the model

In [ ]:
model.fit(batch_size=150, sampler=s, n_iters=2000, bar='notebook')

check out the loss

In [ ]:
plot_loss(model.loss, color='cornflowerblue')

and the solution along with its network approximation

In [ ]:
plot_pair_1d(lambda t: np.sin(2*np.pi*t)+1, model, confidence=0.15, alpha=0.2)

### A more interesting one: poisson equation in $\mathcal{R}^2$ with Dirichlet boundary condition

$$\frac{\partial^2 f}{\partial x^2} + \frac{\partial^2 f}{\partial y^2} = 5\sin[\pi(x + y)];\quad (x, y) \in [0, 1]^2,\ f(0, y)=f(x, 0)=f(1, y)=f(x, 1)=1.
$$

* setting up dimensionality of the problem:

In [ ]:
n_dims = 2

* **Task 1**: implement [laplace operator](https://en.wikipedia.org/wiki/Laplace_operator)

In [ ]:
laplace = lambda u, x, y: D(D(u,x),x) + D(D(u,y),y)

* configuring the equation (by setting the differential form)

In [ ]:
form = lambda u, x, y: laplace(u,x,y)-5*sin(np.pi*(x+y))

* preparing initial/boundary conditions:

In [ ]:
boundary_condition = 1

* **Task 2**: Implement `Sampler` concentrating on the domain-center

In [ ]:
s = NumpySampler('uniform', dim=2)

In [ ]:
plt.imshow(np.histogram2d(*s.sample(1000).T)[0])
plt.colorbar()

In [ ]:
s = s | (NumpySampler('normal', dim=2, scale=0.2) + (.5, .5))

In [ ]:
plt.imshow(np.histogram2d(*s.sample(1000).T)[0])
plt.colorbar()

assemble `config`-dicts for the model

In [ ]:
pde = {'n_dims': n_dims,
       'form': form,
       'boundary_condition': boundary_condition}

config = {'pde': pde}

initialize and train the model-instance

In [ ]:
model = Solver(config)
model.fit(batch_size=150, sampler=s, n_iters=700, bar='notebook')

check out the loss

In [ ]:
plot_loss(model.loss[:], color='cornflowerblue')

In [ ]:
plot_2d(model, 'contourf')

## Getting closer to the inverse problem

Clearly, `PyDEns` is capable of solving PDEs from a wide family. That being said, the key-novelty (and usefulness for oil&gas) of `PyDEns` lies in its ability to perform well in more complex scenarios: (i) solving *parametric families of PDEs* and (ii) *PDEs with trainable coefficients*. 

### Solving parametric family of PDEs

In the previous examples we've used `PyDEns` to solve **one PDE-problem** in **one training-cycle lasting ~1s**. In the upcoming example we'll be using `PyDEns` to solve **parametric family of PDEs** in **one training-cycle lasting ~3s**.

Let us solve simple *parametric family of equations*:

$$
\frac{d f}{d t}= \epsilon \pi \cos[\epsilon \pi t]; \quad t \in [0, 1],\ f(0)=1;
$$
$$
\epsilon - \textrm{parameter}, \quad \epsilon \in [1, 5].
$$

True solution:
$$
\sin[\epsilon \pi t] + 1.
$$

* **Task 3**: wrap up `e`-var into `P`-token to introduce a parameter into the equation

hint: *use the first example* and `P(e)`-structure.

In [ ]:
pde = {'n_dims': 1,
       'form': lambda u, t, e: D(u, t) - ...,
       'initial_condition': 1}

prepare sampler-instance

In [ ]:
s = NumpySampler('uniform')

check out this sampler: it samples points from $\mathcal{R}$:

In [ ]:
s.sample(4)

yet, we have to learn how to solve equation with different values of parameters. As the result, we have to add *another component* to our sampler:

In [ ]:
s = s & NumpySampler('uniform', low=0.5, high=5.5)

assemble config-dicts

In [ ]:
config = {'pde': pde,
          'decay': {'name': 'invtime', 'learning_rate':0.01, 'decay_steps': 100, 'decay_rate': 0.05},
          'track': {'dt': lambda u, t, e: D(u, t)}}

run the training-cycle:

In [ ]:
model = Solver(config)
model.fit(batch_size=4500, sampler=s, n_iters=1500, bar='notebook')

In [ ]:
plot_loss(model.loss, color='cornflowerblue')

In [ ]:
def plot_pair_custom(e):
    solution = lambda t: np.sin(e * np.pi * t) + 1
    points = np.concatenate([np.linspace(0, 1, 200).reshape(-1, 1),
                             e * np.ones((200, 1))], axis=1)
    plt.scatter([0.5], [1.0], marker='o', alpha=0.4, s=70, label=r'$u(0.5)=1$')
    plot_pair_1d(solution, model, points, plot_coord=0, confidence=0.15,
                 title=r'Solution against approximation: $\epsilon=$' + str(np.round(e, 2)))

In [ ]:
_ = interact(plot_pair_custom, e=(1, 5, 0.01))

* **Task 4**: determine value of $\epsilon$ when $u(0.5)=1$

All in all, solving parametric family of PDEs allowed us to determine the value of parameter, that yields the solution satisfying additional constraint.

In the next example we'll take a more direct route to obtain the same result: we'll *train the phase-parameter of an equation* to satisfy the same contraint.

### Solving PDEs with trainable coefficients

In [ ]:
pde = {'n_dims': 1,
       'form': lambda u, t: D(u,t)-V(3.0, 'phase')*np.pi*cos(V(3.0, 'phase')*np.pi*t),
       'initial_condition': 1}

config = {'pde': pde,
          'decay': {'name': 'invtime', 'learning_rate':0.01, 'decay_steps': 100, 'decay_rate': 0.05},
          'track': {'u05': lambda u, t: u - 1},
          'train_steps': {# adjusting u_hat to be better solution aprox with fixed V
                          'uhat_equation': {'scope': '-addendums'},
  
                          # adjusting u_hat to better satisfy additional constraint
                          'uhat_constraint': {'loss': {'name': 'mse', 'predictions': 'u05'}},
                                                    
                          # adjusting V for current u_hat to be better solution approx
                          'v_equation': {'scope': 'addendums'}}}

s1 = NumpySampler('uniform')
s2 = ConstantSampler(0.5)

In [ ]:
model = Solver(config)

In [ ]:
N_ITERS = 20

In [ ]:
model.fit(batch_size=150, sampler=s1, n_iters=2000, train_mode='uhat_equation')

for i in range(N_ITERS):
    model.fit(batch_size=150, sampler=s1, n_iters=100, train_mode='uhat_equation')
    model.fit(batch_size=50, sampler=s2, n_iters=100, train_mode='uhat_constraint')
    model.fit(batch_size=150, sampler=s1, n_iters=100, train_mode='v_equation')

model.fit(batch_size=150, sampler=s1, n_iters=100, train_mode='uhat_equation')

In [ ]:
plot_pair_1d(lambda t, e=3.99: np.sin(e * np.pi * t) + 1, model, confidence=0.2, alpha=0.2)

In [ ]:
model.solve(fetches='phase')

## Solving inverse problems

Moving closer to oil&gaz: solving heat equation in $\mathcal{R} \times \mathcal{R}$

$$\frac{\partial f}{\partial t} - \frac{\partial^2 f}{\partial x^2} = 5x(1 - x);\quad (x, t) \in [0, 1] \times [0, 1],\ f(x, 0) = 4x(1 - x).
$$

* simple heat equation in $\mathcal{R}$

In [ ]:
# describing pde-problem in pde-dict
pde = {'n_dims': 2,
       'form': lambda u, x, t: D(u, t) - D(D(u, x), x) - 5 * x * (1 - x),
       'initial_condition': lambda x: x * (1 - x) * 4}

# put it together in model-config
config = {'pde': pde,
          'decay': {'name': 'invtime', 'learning_rate':0.01, 'decay_steps': 100, 'decay_rate': 0.05}}

# uniform sampling scheme
s = NumpySampler('u', dim=2)

In [ ]:
# train the network on batches of 100 points
model = Solver(config)
model.fit(batch_size=500, sampler=s, n_iters=1200, bar='notebook')

In [ ]:
plot_loss(model.loss, color='cornflowerblue')

In [ ]:
plot_sections_2d(model, timestamps=(0, 0.01, 0.1, 0.3, 0.4, 0.7), ylim=(0.0, 1.0))

What if we additionally know that 

$$
u(x=0.5, t) = \frac{1}{2}\exp^{-2 t} + \frac{1}{2}.
$$

Think: determine **volume of exracted oil** given **pressure dynamics in the well-center**

How does this constraint satisfy for the solution to our equation?

In [ ]:
grid = np.concatenate([0.5 * np.ones((200, 1)),
                       np.linspace(0, 2, 200).reshape(-1, 1)], axis=1)
plot_pair_1d(lambda x: 0.5 * np.exp(-2 * x) + 0.5, model, grid, plot_coord=1)

Not good. Let's fix it!

* not-so-simple heat equation in $\mathcal{R}$ with **unknown** multiplier in rhs:
$$\frac{\partial u}{\partial t} - \frac{\partial^2 u}{\partial x^2} = x(1 - x) * Q(t);\quad (x, t) \in [0, 1] \times [0, 1],\ u(x, 0) = x(1 - x).
$$
and additional constraint on solution-dynamics in $x=0.5$:
$$
u(x=0.5, t) = \frac{1}{2}\exp^{-2 t} + \frac{1}{2}.
$$

In [ ]:
# trainable rhs
block = {'layout': 'fa Rfa fa. fa f',
         'units': [15]*4 + [1],
         'activation': tf.nn.tanh}

# describing pde-problem in pde-dict
pde = {'n_dims': 2,
       'form': lambda u, x, t: D(u, t) - D(D(u, x), x) - 5 * x * (1 - x) * (1 + C(t, 'rhs', **block)),
       'initial_condition': lambda x: x * (1 - x) * 4}

# put it together in model-config
law_at_05 = lambda t: 0.5 * exp(-2 * t) + 0.5
config = {'pde': pde,
          'decay': {'name': 'invtime', 'learning_rate':0.01, 'decay_steps': 100, 'decay_rate': 0.05},
          'track': {'u05': lambda u, x, t: u - law_at_05(t)},
          'train_steps': {# adjusting u_hat to be better solution aprox with fixed rhs
                          'uhat_equation': {'scope': '-addendums'},
  
                          # adjusting u_hat to better satisfy additional constraint
                          'uhat_constraint': {'loss': {'name': 'mse', 'predictions': 'u05'}},
                                                    
                          # adjusting conv-rhs for fixed u_hat to be better solution approx
                          'conv_equation': {'scope': 'addendums'}}}

# sampling schemes
s1 = NumpySampler('u') & (NumpySampler('u') * 2.0)
s2 = ConstantSampler(0.5) & (NumpySampler('u') * 2.0)

In [ ]:
model = Solver(config)

In [ ]:
N_ITERS = 5

In [ ]:
model.fit(batch_size=700, sampler=s1, n_iters=2000, train_mode='uhat_equation')

for i in range(N_ITERS):
    model.fit(batch_size=700, sampler=s1, n_iters=100, train_mode='uhat_equation')
    model.fit(batch_size=100, sampler=s2, n_iters=100, train_mode='uhat_constraint')
    model.fit(batch_size=700, sampler=s1, n_iters=100, train_mode='conv_equation')

model.fit(batch_size=100, sampler=s2, n_iters=10, train_mode='uhat_constraint')

In [ ]:
plot_sections_2d(model, timestamps=(0, 0.05, 0.1, 0.4, 0.6, 0.8), ylim=(0.0, 1.0))

In [ ]:
grid = np.concatenate([0.5 * np.ones((200, 1)),
                       np.linspace(0, 2, 200).reshape(-1, 1)], axis=1)
plot_pair_1d(lambda x: 0.5 * np.exp(-2 * x) + 0.5, model, grid, plot_coord=1,
             title='Constraint against approximation: well center')

That's much better!

In [ ]:
pts = np.linspace(0, 2, 200).reshape(-1, 1)
pts = np.concatenate([np.random.normal(size=(200, 1)), pts], axis=1)

approxs = model.solve(pts, fetches='rhs')
plt.plot(pts[:, 1], approxs, 'r', label='Adjustable coefficient')

plt.xlabel(r'$t$', fontdict={'fontsize': 14})
plt.legend()
plt.grid(True)
plt.show()